In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Snowpark Pandas API
import modin.pandas as spd
# Snowpark pandas modin plugin import
import snowflake.snowpark.modin.plugin

from snowflake.snowpark.context import get_active_session

# snowpark session 생성
session = get_active_session()

In [ ]:
# customer 테이블을 Snowpark Pandas Dataframe 으로 읽어오기
session.use_schema('build_python_de.data')

customer_df = spd.read_snowflake('snowflake_sample_data.tpch_sf1.customer')

st.dataframe(customer_df.head())

In [ ]:
# dataframe 정보
customer_df.info()
print(f"DataFrame shape: {customer_df.shape}")

In [ ]:
customer_df = customer_df['C_CUSTKEY', 'C_NAME', 'C_MKTSEGMENT']

# order 테이블 읽기
spd_order = spd.read_snowflake('snowflake_sample_data.tpch_sf1.orders')

# dataframes 조인
spd_ord_cust = customer_df.merge(spd_order,
                                 left_on='C_CUSTKEY',
                                 right_on='O_CUSTKEY',
                                 how='inner')

# Customer key 기준으로 Aggregations 
column_agg = {
                'O_ORDERKEY':['count'], 
                'O_TOTALPRICE': ['sum' ,'mean']
             }

# Aggregation 적용
spd_agg = spd_ord_cust.groupby(by=['C_CUSTKEY','C_NAME'], 
                                   as_index=False).agg(column_agg)

# 컬럼 remanme
spd_agg.columns = ['C_CUSTKEY', 'C_NAME', 'TOT_ORD',
                   'TOT_ORDER_AMOUNT','AVG_ORDER_AMOUNT']

# 결과를 customer_profile 테이블로 저장
spd_agg.to_snowflake(name='customer_profile',
                     if_exists="replace",
                     index=False)

st.dataframe(spd_agg.head())


In [ ]:
SELECT * FROM customer_profile LIMIT 5;

## Visualize data distribution

Plot histogram distribution for different columns in customer profile

In [ ]:
pd_profile = spd_agg.to_pandas()

fig, axes = plt.subplots(1,4,figsize=(15,3))

colnames = ['TOT_ORD', 'TOT_ORDER_AMOUNT', 'AVG_ORDER_AMOUNT']

for col, ax in zip(colnames, axes.flatten()):
    ax.set_title(col)
    sns.histplot(pd_profile, x=col , ax=ax, kde=True, stat="density", kde_kws=dict(cut=3), alpha=.4, edgecolor=(1, 1, 1, .4))
fig.tight_layout() 


